In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
receipts = pd.read_json('C:\\Users\\gouth\\Desktop\\Fetch_Rewards_Data_Analyst\\initial_data\\receipts.json',lines = True)
brands = pd.read_json('C:\\Users\\gouth\\Desktop\\Fetch_Rewards_Data_Analyst\\initial_data\\brands.json',lines = True)
users = pd.read_json('C:\\Users\\gouth\\Desktop\\Fetch_Rewards_Data_Analyst\\initial_data\\users.json',lines = True)

receipts = receipts.rename(columns = {'_id':'receipt_id'})
users = users.rename(columns = {'_id':'users_id'})
brands = brands.rename(columns = {'_id':'brand_id'})

In [3]:
receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [4]:
#updating the columns where we have inner dictionary in the data.

In [5]:
def date_converter(x):
    return(datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [6]:
receipts.receipt_id = receipts.receipt_id.apply(lambda x : x['$oid'])
receipts.createDate = receipts.createDate.apply(lambda x : x['$date'] )
receipts.createDate = receipts.createDate.apply(date_converter)
receipts.dateScanned = receipts.dateScanned.apply(lambda x : x['$date'] )
receipts.dateScanned = receipts.dateScanned.apply(date_converter)

#containins null values so getting those non null values and updating the inner dictionary values
receipts.finishedDate.loc[receipts.finishedDate.notnull()] = receipts.finishedDate[receipts.finishedDate.notnull()].apply(lambda x : x['$date'])
receipts.finishedDate.loc[receipts.finishedDate.notnull()] = receipts.finishedDate[receipts.finishedDate.notnull()].apply(date_converter)
receipts.modifyDate = receipts.modifyDate.apply(lambda x : x['$date'])
receipts.modifyDate = receipts.modifyDate.apply(date_converter)
receipts.pointsAwardedDate.loc[receipts.pointsAwardedDate.notnull()] = receipts.pointsAwardedDate[receipts.pointsAwardedDate.notnull()].apply(lambda x : x['$date'] )
receipts.pointsAwardedDate.loc[receipts.pointsAwardedDate.notnull()] = receipts.pointsAwardedDate[receipts.pointsAwardedDate.notnull()].apply(date_converter) 
receipts.purchaseDate.loc[receipts.purchaseDate.notnull()] = receipts.purchaseDate[receipts.purchaseDate.notnull()].apply(lambda x : x['$date'])
receipts.purchaseDate.loc[receipts.purchaseDate.notnull()] = receipts[receipts.purchaseDate.notnull()].purchaseDate.apply(date_converter)

 - creating a seperate table for receipt item list   

In [7]:
#assigining each element in the list with the appropirate receipt_id so that we can join and get the information.
l = []
for i in receipts.itertuples(index=True, name='Pandas'):
    if type(i.rewardsReceiptItemList) != float:
        for j in i.rewardsReceiptItemList:
            j['receipt_id'] = i.receipt_id
            l.append(j)

#creating a seperate table for receipt item list            
receiptItemList = pd.DataFrame(l)

In [8]:
receipts = receipts.drop(columns = ['rewardsReceiptItemList'])

- Similarly for users table

In [9]:
users.users_id = users.users_id.apply(lambda x : x['$oid'])
users.createdDate = users.createdDate.apply(lambda x : x['$date'] )
users.createdDate = users.createdDate.apply(date_converter)
users.lastLogin.loc[users.lastLogin.notnull()] = users.lastLogin[users.lastLogin.notnull()].apply(lambda x : x['$date'])
users.lastLogin.loc[users.lastLogin.notnull()] = users.lastLogin[users.lastLogin.notnull()].apply(date_converter)

In [10]:
users.head()

,users_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


- Brands table

In [11]:
brands.head()

,brand_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [12]:
brands.brand_id = brands.brand_id.apply(lambda x : x['$oid'])
#converting cpg columns to two sub columns cpg reference and cpg id 
brands[['cpg.$ref','cpg.id.oid']] = pd.json_normalize(brands['cpg'])

In [13]:
brands.to_csv('brands.csv',index = False)
users.to_csv('users.csv',index = False)
receipts.to_csv('receipts.csv',index = False)
receiptItemList.to_csv('receiptItemList.csv',index = False)

In [14]:
receiptItemList.shape

(6941, 35)

In [15]:
receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaN,2021-01-03 15:25:42,NaN,5.0,2021-01-03 00:00:00,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


# Second: Write a query that directly answers a predetermined question from a business stakeholder

In [16]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [17]:
users.to_sql('users', con=engine)
brands.drop(columns = ['cpg'],axis =1).to_sql('brands',con = engine)
receipts.to_sql('receipts',con = engine)
receiptItemList.to_sql('receiptItemList',con = engine)

### 1) What are the top 5 brands by receipts scanned for most recent month?

- For the receipts in the most recent month the brand value is none, that is why the results are none. I have written generic query which can be used.

In [18]:
engine.execute("select distinct(b.name),count(b.brandCode) as count_bc from brands b \
               inner join \
               (SELECT brandCode from receiptItemList where receipt_id in \
               (SELECT r.receipt_id from receipts r \
               where  strftime('%Y-%m',dateScanned) =(SELECT MAX(strftime('%Y-%m',dateScanned)) FROM receipts)) \
              ) r  \
               on r.brandCode = b.brandCode group by b.brandCode order by count_bc desc LIMIT 5;").fetchall()

[]

### 2) How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [19]:
engine.execute("\
               SELECT brandCode from receiptItemList where brandCode IS NOT NULL and  receipt_id in \
               (SELECT r.receipt_id from receipts r \
               where  strftime('%Y-%m',dateScanned) =(SELECT distinct(strftime('%Y-%m',dateScanned)) FROM receipts order by 1 desc LIMIT 1,1) \
              )").fetchall()

[('BRAND',), ('BRAND',), ('MISSION',), ('VIVA',), ('BRAND',), ('MISSION',)]

In [20]:
engine.execute("select distinct(b.name),count(b.brandCode) as count_bc from brands b \
               inner join \
               (SELECT brandCode from receiptItemList where brandCode IS NOT NULL and  receipt_id in \
               (SELECT r.receipt_id from receipts r \
               where  strftime('%Y-%m',dateScanned) =(SELECT distinct(strftime('%Y-%m',dateScanned)) FROM receipts order by 1 desc LIMIT 1,1) \
              )) r  \
               on r.brandCode = b.brandCode group by b.brandCode order by count_bc desc LIMIT 5;").fetchall()

[('Viva', 1)]

### 3) When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [21]:
engine.execute("select rewardsReceiptStatus,avg(totalSpent) as average_spend from receipts\
               where rewardsReceiptStatus == 'REJECTED' \
            or rewardsReceiptStatus == 'FINISHED' group by rewardsReceiptStatus ").fetchall()

[('FINISHED', 80.85430501930502), ('REJECTED', 23.326056338028184)]

In [22]:
engine.execute("select rewardsReceiptStatus,avg(totalSpent) as average_spend from receipts\
               where rewardsReceiptStatus == 'REJECTED' \
            or rewardsReceiptStatus == 'FINISHED' group by rewardsReceiptStatus order by average_spend desc LIMIT 1 ").fetchall()

[('FINISHED', 80.85430501930502)]

- FINISHED(ACCEPTED) has more average spend compared with rejected.

### 4)When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [23]:
engine.execute("select rewardsReceiptStatus,sum(purchasedItemCount) as average_spend from receipts\
               where rewardsReceiptStatus == 'REJECTED' \
            or rewardsReceiptStatus == 'FINISHED' group by rewardsReceiptStatus ").fetchall()

[('FINISHED', 8184.0), ('REJECTED', 173.0)]

In [24]:
engine.execute("select rewardsReceiptStatus,sum(purchasedItemCount) as average_spend from receipts\
               where rewardsReceiptStatus == 'REJECTED' \
            or rewardsReceiptStatus == 'FINISHED' group by rewardsReceiptStatus order by average_spend desc LIMIT 1 ").fetchall()

[('FINISHED', 8184.0)]

- FINISHED(ACCEPTED) has more number of items purchased compared with rejected.

- Which brand has the most spend among users who were created within the past 6 months?

### 5) Which brand has the most spend among users who were created within the past 6 months

In [25]:
#as there are no users in the past 6 months from the date 9/30/21.
#I am considering the latest date of user creation till 6 months before that.

In [26]:
engine.execute("select r.brandCode,sum(b.totalSpent) as most_spend from receiptItemList r \
inner join \
               (select receipt_id,totalSpent from receipts where userId in \
(select users_id from users where createdDate > \
(select date(MAX(createdDate),'-6 month') from users)\
)) b \
on r.receipt_id = b.receipt_id where r.brandCode IS NOT NULL group by r.brandCode order by most_spend desc LIMIT 1 ").fetchall()

[('BEN AND JERRYS', 197337.68000000025)]

-  BEN AND JERRYS brand has the most spend among users who were created within the past 6 months

### 6) Which brand has the most transactions among users who were created within the past 6 months?

In [27]:
#as there are no users in the past 6 months from the date 9/30/21.
#I am considering the latest date of user creation till 6 months before that.

In [28]:
engine.execute("select r.brandCode,count(b.receipt_id) as most_spend from receiptItemList r \
inner join \
               (select distinct(receipt_id) from receipts where userId in \
(select users_id from users where createdDate > \
(select date(MAX(createdDate),'-6 month') from users)\
)) b \
on r.receipt_id = b.receipt_id where r.brandCode IS NOT NULL group by r.brandCode order by most_spend desc LIMIT 1").fetchall()

[('HY-VEE', 291)]

- HY-VEE brand has the most transactions among users who were created within the past 6 months

# Third: Evaluate Data Quality Issues in the Data Provided

In [29]:
def print_missing(some_df):
    x = pd.DataFrame([some_df.shape[0] - np.array(list(some_df.isnull().sum())),list(some_df.isnull().sum()),np.array(list(some_df.isnull().sum())) + some_df.shape[0] - np.array(list(some_df.isnull().sum()))],columns = some_df.columns).transpose()
    x.columns = ['available row count','missing value count','total row count']
    return x

In [30]:
print_missing(brands)

,available row count,missing value count,total row count
brand_id,1167,0,1167
barcode,1167,0,1167
category,1012,155,1167
categoryCode,517,650,1167
cpg,1167,0,1167
name,1167,0,1167
topBrand,555,612,1167
brandCode,933,234,1167
cpg.$ref,1167,0,1167
cpg.id.oid,1167,0,1167


In [31]:
brands[brands.categoryCode.isna()]

,brand_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.id.oid
7,5cdad0f5166eb33eb7ce0faa,511111104810,Condiments & Sauces,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '559c2234e4b0...",J.L. Kraft,NaN,J.L. KRAFT,Cogs,559c2234e4b06aca36af13c6
8,5ab15636e4b0be0a89bb0b07,511111504412,Canned Goods & Soups,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5a734034e4b0...",Campbell's Home Style,0.0,CAMPBELLS HOME STYLE,Cogs,5a734034e4b0d58f376be874
9,5c408e8bcd244a1fdb47aee7,511111504788,Baking,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '59ba6f1ce4b0...",test,NaN,TEST,Cogs,59ba6f1ce4b092b29c167346
11,57c08106e4b0718ff5fcb02c,511111102540,NaN,NaN,"{'$ref': 'Cpgs', '$id': {'$oid': '5332f5f2e4b0...",MorningStar,NaN,NaN,Cpgs,5332f5f2e4b03c9a25efd0aa
12,588ba07be4b02187f85cdadd,511111201076,Baking,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '559c2234e4b0...",Calumet,0.0,CALUMET,Cogs,559c2234e4b06aca36af13c6
...,...,...,...,...,...,...,...,...,...,...
1159,585a96cbe4b03e62d1ce0e88,511111501619,Beverages,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5fbe4b0...",Pepsi Max,0.0,,Cogs,5332f5fbe4b03c9a25efd0ba
1160,5887a216e4b02187f85cdad5,511111401155,Deli,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '559c2234e4b0...",Claussen,0.0,CLAUSSEN,Cogs,559c2234e4b06aca36af13c6
1161,5332f709e4b03c9a25efd0f2,511111403845,Beer Wine Spirits,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f709e4b0...",Blue Moon,0.0,BLUE MOON,Cogs,5332f709e4b03c9a25efd0f1
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc


- We see categoryCode are having more nulls when compared with category columns. This might become inconsistency If we want to join table based on categorycode and also some categories such as Beverages,Baking have NaN's though there are some values where they have categorycode which is very serious issue, since number of rows retrieved when using categorycode filter will differ when using with category.

In [32]:
#lets see brandCode values and receiptItemList brandcode values
len(brands[brands.brandCode.notnull()].brandCode.unique())

897

In [33]:
len(receiptItemList[receiptItemList.brandCode.notnull()].brandCode.unique())

227

In [34]:
for i in receiptItemList[receiptItemList.brandCode.notnull()].brandCode.unique():
    if i not in brands[brands.brandCode.notnull()].brandCode.unique():
        print(i)

MISSION
BRAND
KRAFT EASY CHEESE
WINGSTOP
GERM-X
BEN AND JERRYS
BORDEN
KLARBRUNN
HY-VEE
LIGHT & FIT GREEK
CONNIE'S PIZZA
VAN DE KAMP'S
HATCH FARMS
KELLOGG'S
TEMPTATIONS
NATURE'S PATH ORGANIC
DOLE
EL MONTEREY
BIGELOW
HY-VEE SELECT
KIKKOMAN
SPECIAL K
HILLSHIRE FARM
JUST BARE
LAURA'S LEAN BEEF
CAL-ORGANIC FARMS
BUSH'S BEST
FOLGERS
KASHI
LIPTON
GREEN GIANT
HARVEST SNAPS
THAT'S SMART!
ADVIL
CHICKEN OF THE SEA
STARKIST
TIC TAC
SO DELICIOUS
WONDERFUL
LIGHT & FIT
HANOVER
HIDDEN VALLEY
DANNON
FAGE
ORAL-B GLIDE
CAMPBELL'S
FRENCH'S
CRISPIX
KING ARTHUR FLOUR
KITCHEN BASICS
MCCORMICK
OLD EL PASO
ZESTA
AZTECA
BUNNY
HONEY BUNCHES OF OATS
SIMPLE TRUTH ORGANIC
BOTA BOX
DARE
LINDT
ORGANIC ROOT STIMULATOR
MERKT'S
MORTON
FRONTERA
KARO
CHEESE
FLORIDA'S NATURAL
BLUE DIAMOND
LUNDBERG FAMILY FARMS
NUTRI-GRAIN
DIGIORNO
KEMPS
THAI KITCHEN
THOMAS'
POWER CRUNCH
GERBER
ORGANICVILLE
PURINA ONE
DR PEPPER
C&H
CHEERIOS
SIMPLE TRUTH
STOUFFER'S
PRINGLES
MCCORMICK GRILL MATES
THOMAS ENGLISH MUFFINS
MARTINELLI'S
MARIE CALL

- out of 227 unique brands in recieptItemList there are lot of brands that dont have information in brands table which is a problem.
- brands table is inconsistent in brandCOde and categoryCode columns.

In [35]:
print_missing(users)

,available row count,missing value count,total row count
users_id,495,0,495
active,495,0,495
createdDate,495,0,495
lastLogin,433,62,495
role,495,0,495
signUpSource,447,48,495
state,439,56,495


In [36]:
len(users.users_id.unique())

212

In [37]:
#This suggests that the every row in users table is not a unique (uuid). Lets see why

In [38]:
users[users.users_id == '5ff1e194b6a9d73a3a9f1052']

,users_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
10,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
12,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
14,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
15,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


- Lot of duplicate rows does not provide any clear idea. Which is serious data quality issue.

In [39]:
users.role.unique()
#it was mentioned in the data dictionary role is constant value set to 'CONSUMER' but I can see there are more than one role
#in the data.The data dictionary is not clear in this.

array(['consumer', 'fetch-staff'], dtype=object)

In [40]:
print_missing(receipts)

,available row count,missing value count,total row count
receipt_id,1119,0,1119
bonusPointsEarned,544,575,1119
bonusPointsEarnedReason,544,575,1119
createDate,1119,0,1119
dateScanned,1119,0,1119
finishedDate,568,551,1119
modifyDate,1119,0,1119
pointsAwardedDate,537,582,1119
pointsEarned,609,510,1119
purchaseDate,671,448,1119
